In [7]:
import pandas as pd
import datetime
x = datetime.datetime.now()
# merge = pd.read_csv('../data/merged/'+str(x.strftime("%Y%m%d"))+'merge_withbaselineSymbol-withoutMints.csv')
# merge = pd.read_csv('../data/merged/20220610merge_withbaselineSymbol-withoutMints.csv')
merge = pd.read_csv('../clustering/mergewithBaseline.csv')

In [5]:
def selectTopRows(merge,valueCounts,topRows=50):
    mostUsedNFT = valueCounts[0:topRows].keys()
    return merge[merge["SymbolNFTAddress"].isin(mostUsedNFT)]

In [17]:
import numpy as np
import pandas as pd

def currencyTranslation(value, date, priceTable):
    return priceTable[priceTable["timestamp"] == date].iloc[0,1:4].mean() * value

def convertNFTTradesGroupedData(groupedData):
    print("Start converting data")
    priceTable = pd.read_csv('../data/Dollar/ETH-USD.csv')
    priceTable['timestamp'] = pd.to_datetime(priceTable['Date'], utc=True)

    grouped = []
    row=0

    for (TransactionName, groupobject) in groupedData.__iter__():
        row += 1
        if row % 100000 == 0 : print(row)
        if row % 1000000 == 0 : 
            print("save")
            grouped_save = pd.DataFrame(grouped)
            grouped_save.to_csv('20220615_total.csv', index=False)
            print("save complete")

        groupedNFTS = groupobject.groupby(["baseLine"])
        
        totalLength = len(groupobject)
        etherPrice = np.array(groupobject["transactionEther"])[0]
        timeStampDay= np.array(groupobject["timestamp_day"])[0]
        timeStamp = np.array(groupobject["timestamp"])[0]
        # timeStampHour = np.array(groupobject["timestamp_hour"])[0]
        
        for (NFTname, groupobNFTs) in groupedNFTS.__iter__():

            adjDollarPrice = (currencyTranslation(etherPrice, timeStampDay, priceTable))/totalLength
            logadjDollarPrice = 0
            if adjDollarPrice != 0 :
                logadjDollarPrice = np.log10(adjDollarPrice)
            
            grouped.append({
                "timeStamp":timeStamp,
                "transactionEther":etherPrice,
                "transactionEther_average":etherPrice/totalLength,
                "priceInDollar": adjDollarPrice,
                "priceInDollarLog": logadjDollarPrice,
                "quant": len(groupobNFTs),
                "nft_address": NFTname,
                "transaction":TransactionName
            })
        
    return pd.DataFrame(grouped)

In [18]:
total = merge.groupby(["transaction_hash"])

In [19]:
# EXPECT 100 minutes
grouped = convertNFTTradesGroupedData(total)

Start converting data
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
save
save complete
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
save
save complete
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
save
save complete
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
save
save complete
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
save
save complete
5100000


In [20]:
grouped.to_csv('20220615_total.csv', index=False)

In [21]:
grouped.to_csv('../clustering/20220615_total.csv', index=False)

In [11]:
# pivot_mean = pd.pivot_table(grouped, values="priceInDollar", index="timeStamp-hour", columns="nft_address", aggfunc=np.mean)
# pivot_difference = pd.pivot_table(grouped, values="priceInDollar", index="timeStamp-hour", columns="nft_address", aggfunc=np.mean).rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
# pivot_difference_log = pd.pivot_table(grouped, values="priceInDollarLog", index="timeStamp-hour", columns="nft_address", aggfunc=np.mean).rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
# pivot_mean_log = pd.pivot_table(grouped, values="priceInDollarLog", index="timeStamp-hour", columns="nft_address", aggfunc=np.mean)
# pivot_max_log = pd.pivot_table(grouped, values="priceInDollarLog", index="timeStamp-hour", columns="nft_address", aggfunc=np.max)
# pivot_min_log = pd.pivot_table(grouped, values="priceInDollarLog", index="timeStamp-hour", columns="nft_address", aggfunc=np.min)
# pivot_std_log = pd.pivot_table(grouped, values="priceInDollarLog", index="timeStamp-hour", columns="nft_address", aggfunc=np.std)